In [1]:
data_partition = {
        "gwilliams2023": {
            "testing_subjects": [19, 20, 21],
            "testing_tasks": [0],
        },
        "schoffelen2022": {
            "testing_subjects": [],
            "testing_tasks": [8, 9],
        },
}

In [2]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=None
#     )

In [3]:
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=512,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=3,
    layout_proj=False,
    layout_scaling="midpoint",
    # Merger with spatial attn
    merger=True,
    merger_emb_type="linear",
    merger_emb_dim=512,
    merger_channels=256,
    merger_dropout=True,
    merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=True,
    conditional_layers_dim='hidden_dim',  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input='continuous',
    transformer_encoder_emb='groupconv',
    transformer_encoder_layers=4,
    transformer_encoder_heads=8,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

In [4]:
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={'all': (0.5, 80)},
    max_random_shift=1.0,
    window_size= 4,
    window_stride=1,
    brain_clipping=20,
    baseline_window=None,
    notch_filter=True,
    scaling='both',
    delay=0.15,
    # Hyperparameters
    learning_rate = 5e-4,
    weight_decay = 1e-4,
    epochs = 50,
    batch_size = 256,
    use_clip_loss = True,
    use_mse_loss = True,
    alpha =0.6,
    random_test_size =20,
    seed = 42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: 'audio' for study in data_partition.keys()},
    data_path='data',
    save_path='saves/Dec18_CLIP_MSE_full_both_datasets',
    clear_cache=False,
)

Loading Gwilliams2023 with batch type audio
Loading Schoffelen2022 with batch type audio
Data partitioned on studies ['gwilliams2023', 'schoffelen2022'].
Train: 165, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

Conditional layer study initialized with 3 conditions
Conditional layer subject initialized with 31 conditions
TransformerEncoder initialized with 4 layers, 512 d_model, 8 nhead
	Embedding: groupconv, params: 12612096
SimpleConv initialized with 27678592 parameters, cond: ['study', 'subject']
Merger True, merger channels 256
ConvBlocks: 6, hidden_dim: 512, params 14170112


In [12]:
try:
    session.train(
        device='cuda',
        buffer_size=30,
        num_workers=multiprocessing.cpu_count() - 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print('Exited')

2024-12-19 09:07:55,139	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1:  14%|█▍        | 23/165 [12:21<1:16:16, 32.23s/it]

Exited


In [ ]:
# import os

# def delete_file_recursive(directory, filename):
#     for root, dirs, files in os.walk(directory):
#         if filename in files:
#             file_path = os.path.join(root, filename)
#             try:
#                 os.remove(file_path)
#                 print(f"Deleted: {file_path}")
#             except PermissionError:
#                 print(f"Permission denied: {file_path}")
#             except Exception as e:
#                 print(f"Error deleting {file_path}: {e}")

# directory = os.getcwd()
# filename = 'training_log.log'
# delete_file_recursive(directory, filename)